In [1]:
pip install datasets transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

=
dataset = load_dataset("SKNahin/bengali-transliteration-data")


dataset = dataset["train"].train_test_split(test_size=0.2)  # 80/20 split
train_data = dataset["train"]
val_data = dataset["test"]

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")


Select Model


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load mT5-small model and tokenizer
model_name = "google/mt5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Data processing

In [4]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 4004
    })
    test: Dataset({
        features: ['bn', 'rm'],
        num_rows: 1002
    })
})


In [5]:
print(train_data.column_names)


['bn', 'rm']


In [6]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")


def preprocess_data(batch):

    inputs = ["translate Banglish to Bangla: " + text for text in batch["rm"]]
    targets = batch["bn"]


    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


train_data = train_data.map(preprocess_data, batched=True)
val_data = val_data.map(preprocess_data, batched=True)

# Convert to PyTorch tensors
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])




Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

Train

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./mt5-banglish-bangla",  # Save directory
    run_name="my_custom_run_name",  # Set a custom run name here
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save at the end of each epoch
    learning_rate=5e-5,                    # Learning rate
    per_device_train_batch_size=16,        # Batch size
    per_device_eval_batch_size=16,
    num_train_epochs=5,                    # Number of epochs

    weight_decay=0.01,                     # Weight decay for regularization
    save_total_limit=2,                    # Save only the last 2 checkpoints
    logging_dir="./logs",                  # Directory for logs
    logging_steps=500,                     # Log every 500 steps
    load_best_model_at_end=True,           # Load the best model at the end of training
    metric_for_best_model="loss",          # Use loss for best model selection
    greater_is_better=False,               # Lower loss is better
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Trainer


In [8]:
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.2 MB/s eta 0:00:00


In [9]:
import evaluate
import numpy as np


metric = evaluate.load("sacrebleu")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Replace -100 in labels (due to padding) with an empty string
    decoded_labels = [[label] for label in decoded_labels]

    # Compute BLEU score
    bleu_score = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": bleu_score["score"]}


tuning

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,

)

In [11]:
trainer.train()
#trainer.evaluate()
trainer.save_model("./fine_tuned_mt5")
tokenizer.save_pretrained("./fine_tuned_mt5")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.63 GiB. GPU 0 has a total capacity of 14.75 GiB of which 4.86 GiB is free. Process 10548 has 9.89 GiB memory in use. Of the allocated memory 9.66 GiB is allocated by PyTorch, and 108.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Inference with Fine-Tuned Model

In [14]:
def translate_text(text):

    input_text = "translate Banglish to Bangla: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)


    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

result = translate_text("ami bhalo achi")
print("Translation:", result)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)